## Libraries

In [21]:
import numpy as np
import random
import collections
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName("MyApp").getOrCreate()
sc = spark.sparkContext

## Classes

In [6]:
class Shingling:
  """A class Shingling that constructs k–shingles of a given length k (e.g., 10) from a given document, 
  computes a hash value for each unique shingle, 
  and represents the document in the form of an ordered set of its hashed k-shingles."""
  
  def __init__(self, doc):
    self.c = 2**32-1 #Mersenne prime
    self.a = 7
    self.b = 10
    self.k = 5
    self.shingles = set([doc[i:i+self.k] for i in range(len(doc))])
    self.hashed_shingles = set([self.hashing(i) for i in self.shingles])
  
  def hashing(self, x):
    x = int.from_bytes(x.encode(), "little")
    return (self.a * x + self.b) % self.c


In [7]:
class CompareSets:
  """A class CompareSets that computes the Jaccard similarity of two sets of integers 
  – two sets of hashed shingles."""
  
  def __init__(self, set_x, set_y):
    intersection = len(list(set_x.intersection(set_y)))
    union = (len(set_x) + len(set_y)) - intersection
    self.jsim =  float(intersection) / union


In [8]:
class MinHashing:
  """A class MinHashing that builds a minHash signature (in the form of a vector or a set)
   of a given length n from a given set of integers (a set of hashed shingles)."""
  def __init__(self, sets):

    
    self.c = 2**32-1 #Mersenne prime
    self.k = 100
    hash_list = [self.hashing for i in range(self.k)]
    
    #Unionize all the available shingles from the documents
    
    self.union_set = list(set().union(*sets))

    I = np.zeros((len(self.union_set), len(sets)))
    self.S = np.ones((self.k, len(sets))) * float("Inf")

    #Making the Input Matrix
    for c in range(len(sets)):
      for r in range(len(self.union_set)):
        if self.union_set[r] in sets[c]:
          I[r,c] = 1


    #Making the Signature Matrix

    for r in range(I.shape[0]):
      computed_hash = list()
      for hash_func in hash_list:
        computed_hash.append(hash_func(r))
      for c in range(I.shape[1]):
        if I[r,c] == 1:
          for k in range(len(computed_hash)):
            if computed_hash[k] < self.S[k,c]:
              self.S[k,c] = computed_hash[k]



  def hashing(self, x):
    a = random.randint(1,1000)
    b = random.randint(1,1000)

    return (a * x + b) % self.c


In [9]:
class CompareSignatures:
  """A class CompareSignatures that estimates similarity of two integer vectors –
   minhash signatures – as a fraction of components, in which they agree."""
  def __init__(self, vec1, vec2):
    match = 0
    for i in range(len(vec1)):
      if vec1[i] == vec2[i]:
        match += 1
    self.sim = match / len(vec1)


In [25]:
class LSH:
  """ A class LSH that implements the LSH technique: 
  given a collection of minhash signatures (integer vectors) and a similarity threshold t, 
  the LSH class (using banding and hashing) finds candidate pairs of signatures agreeing 
  on at least fraction t of their components."""
  def __init__(self, S, t, b=20):
    #Initialize band size...
    n , d = S.shape
    r = n/b
    t_th = (1/b)**(1/r)
    assert( n % b == 0)
    self.k_buckets = 2**32

    bands = np.array_split(S, b, axis=0)
    buckets = collections.defaultdict(list)
    # Iterating through bands and hash to buckets
    for i in range(b):
      for j in range(d):

        hash_val = self.hash_func(bands[i][:,j])
        buckets[str(hash_val) + "," + str(i)].append(j)


    matches = collections.defaultdict(int)
    self.c_pairs = list()
    
    # Find candidate pairs
   
    for key, bucket in buckets.items():
      if len(bucket)> 1:
        for idx in bucket:
          for jdx in bucket:
            if idx != jdx:

              matches[tuple(sorted((idx, jdx)))] += 1

    for key, value in matches.items():
      if value/b >=t:
        if key not in self.c_pairs:
          self.c_pairs.append(key)
  def hash_func(self, v):
    
    nr = hash(tuple(list(v)))
    return nr % self.k_buckets


To test and evaluate scalability (the execution time versus the size of input dataset) of your implementation, write a program that uses your classes to find similar documents in a corpus of 5-10 documents. Choose a similarity threshold s (e.g., 0,8) that states that two documents are similar if the Jaccard similarity of their shingle sets is at least s. 


## Testing

In [26]:
def opendoc(name):
  with open(name) as f:
      doc = f.read()
  return doc

In [27]:
ds = []
ogs = []
df = pd.read_csv("content.csv")
for _,c in df.head(100)["content"].iteritems():
  ogs.append(c)
  ds.append(Shingling(c))


## Compare Sets

In [28]:
docs = list()
og_docs = list()
idxs = set()
for i in range(len(ds)):
  for j in range(len(ds)):
    similar = CompareSets(ds[i].hashed_shingles, ds[j].hashed_shingles).jsim
    if 0.4 < similar and i != j:
      if i not in idxs: 
        docs.append(ds[i])
        og_docs.append(ogs[i])
      if j not in idxs:
        docs.append(ds[j])
        og_docs.append(ogs[j])
      idxs.add(i)
      idxs.add(j)

In [29]:
CompareSets(docs[0].hashed_shingles, docs[1].hashed_shingles).jsim

0.5400192864030858

## Compare Signatures

In [30]:
sets = list()
for d in docs:
  sets.append(d.hashed_shingles)

In [31]:
print(len(sets))

31


In [32]:
sig_M = MinHashing(sets).S

In [33]:
CompareSignatures(sig_M[:,0], sig_M[:,1]).sim

0.65

## Find candidate Pairs

In [36]:
for a, b in LSH(sig_M, 0.8).c_pairs:
  jacc_sim = CompareSets(docs[a].shingles, docs[b].shingles).jsim
  pairs[(a,b)] = jacc_sim
  if jacc_sim < 0.8:
    print("False Positive")
  print("Documents:", a,b)
  print("Jaccard Similarity:", jacc_sim)
  print("------")

Documents: 9 10
Jaccard Similarity: 0.8316326530612245
------
Documents: 9 11
Jaccard Similarity: 0.8994413407821229
------
Documents: 10 11
Jaccard Similarity: 0.8214285714285714
------
False Positive
Documents: 14 15
Jaccard Similarity: 0.7439483593329748
------
Documents: 16 18
Jaccard Similarity: 0.8384321223709369
------
Documents: 19 20
Jaccard Similarity: 0.8886966551326413
------
Documents: 27 28
Jaccard Similarity: 0.8395522388059702
------
Documents: 29 30
Jaccard Similarity: 0.8568453930244664
------
False Positive
Documents: 3 4
Jaccard Similarity: 0.7941888619854721
------
Documents: 16 17
Jaccard Similarity: 0.8255144032921811
------
False Positive
Documents: 17 18
Jaccard Similarity: 0.7482935153583617
------
